# Test Calchylus 3 module

### System settings

In [1]:
(import sys hy calchylus3)

(print 'Python 'version: sys.version)
(print 'Hy 'version: hy.__version__)
(print 'Calchylus 'version: calchylus3.__version__)

(assert (= hy.__version__ "0.15.0"))
(assert (= calchylus3.__version__ "0.1.4"))

Python version: 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
Hy version: 0.15.0
Calchylus version: 0.1.4


[None, None, None, None, None, None]

### Import calchylus 3

In [2]:
(require [calchylus3.lambdas [*]])

[None]

### Initializers

#### Without macros

In [3]:
; first parameter is the binder, second is the delimitter when printing a lambda form
(without-macros λ \.)
; lambda function should be available now
(assert (callable Function))
; lambda form should be a callable function
(assert (callable (λ x x)))
; lambda form callable function string representation
(assert (= (str (λ x x)) "(λ x . x)"))

[<function <lambda> at 0x0000013CD83D3950>, None, None, None]

Initialize a new variable with a lambda function

In [4]:
(setv x (λ x x))
(assert (callable x))
(assert (= (str x) "(λ x . x)"))
(assert (= (x 1) 1))
; remove x from workspace from further disturbing lambda expression variables
(del x)

[None, None, None, None, None]

If function body uses a variable that is not defined as a function argument, then name error is thrown.

In [5]:
(try
  (λ x y)
  (except [e NameError]
    (assert (= (str (λ x 'y)) "(λ x . y)"))))

[None]

Similar form with different variable names should return same value

In [6]:
(assert (= (str (λ x x)) "(λ x . x)"))
(assert (= (str (λ y y)) "(λ y . y)"))
(assert (= ((λ x x) 1) ((λ y y) 1)))

[None, None, None]

With macros

In [7]:
(with-macros L)

[None]

### Constant

In [8]:
; constant without parameter and body 
(assert (= (CONST) None))
; constant without parameter. body must be quoted!
(try
  (CONST x)
  (except [e NameError]
    (assert (= (CONST 'x) 'x))))
; constant with body
(assert (= (str (CONST x 1)) "(L x 1)"))
; constant with body and value is same with any value
(assert (= ((CONST x 1) 1) ((CONST y 1) 2)))

[None, None, None, None]

### Identity

In [9]:
; identity, without an argument
(assert (= (str IDENT) "(L x x)"))
; identity, with an argument
(assert (= (IDENT 1) 1))
; identity, nested
(assert (= (IDENT (IDENT 1)) 1))

[None, None, None]

### Booleans

Not, true, false

In [10]:
(assert (= (, (str TRUE) (TRUE 1 0)
              (str FALSE) (FALSE 1 0))
           (, "(L a (L b a))" 1
              "(L a (L b b))" 0)))

(assert (= (TRUE 0 1) 0))
(assert (= (FALSE 0 1) 1))
(assert (= ((NOT FALSE) 0 1) (TRUE 0 1)))

[None, None, None, None]

### Condition with booleans

In [11]:
(assert (= (str COND) "(L a (L b (L c (a b c))))"))
(assert (= (COND TRUE 'a 'b) 'a))
(assert (= (COND FALSE 'a 'b) 'b))

[None, None, None]

### 2-tuples / pairs

In [12]:
; nil
(assert (= (str NIL) "(L x (L a (L b a)))"))
; empty, list end
(assert (= (str EMPTY) "(L s (s (L x (L a (L b a))) (L x (L a (L b a)))))"))
; pair constructor
(assert (= (str (PAIR TRUE NIL)) "(L s (s (L a (L b a)) (L x (L a (L b a)))))"))
; selector, true
(assert (= (str (PAIR 'T 'F TRUE)) 'T))
; selector, false
(assert (= (str (PAIR 'T 'F FALSE)) 'F))
; head and tail selectors
(assert (= (, (str (HEAD (PAIR TRUE NIL)))
              (str (TAIL (PAIR TRUE NIL))))
           (, "(L a (L b a))" "(L x (L a (L b a)))")))
; nested pairs and heads and tails
(assert (=
   (, (str (HEAD (PAIR TRUE (PAIR TRUE NIL))))
      (str (HEAD (TAIL (PAIR TRUE (PAIR TRUE NIL)))))
      (str (TAIL (TAIL (PAIR TRUE (PAIR TRUE NIL))))))
   (, "(L a (L b a))" "(L a (L b a))" "(L x (L a (L b a)))")))
; simple condition
(assert (=
    (, (str (COND TRUE TRUE FALSE) )
       (str (COND FALSE TRUE FALSE)))
    (, "(L a (L b a))" "(L a (L b b))")))

[None, None, None, None, None, None, None, None]

### Let and Let*

In [21]:
(assert (= (repr* (LET a 1 b 2 d 3 [a b 'c d])) "(1 2 c 3)"))
(assert (= (repr* (LET a 1 [a a ((L a a) 2)])) "(1 1 2)"))
(assert (= (repr* (LET a 1 b 'a [a b])) "(1 a)"))
(assert (= (repr* (LET a 1 c ['a 'b] (LET b a c))) "(a b)"))
(assert (= (repr* (LET* a 1 c [a 'b] (LET b a c))) "(1 b)"))
(assert (= (repr* (LET* a 1 b a c [a b] c)) "(1 1)"))
(assert (= (repr* (LET* a 1 c (LET b a [a b]) c)) "(1 1)"))
(assert (= (repr* (LET a 1 b ['a 'b] b)) "(a b)"))
(assert (= (repr* (LET a 1 (LET b a [a b]))) "(1 1)"))
(assert (= (repr* (LET* a 1 b a [a b])) "(1 1)"))
(try
  (LET x)
  (except [e NameError]))
(try
  (LET x 1)
  ; 'int' object is not callable
  (except [e TypeError]))
;(assert (= (repr (LET* x)) "((L x x))"))
;(assert (= (repr (LET* x 1)) "((L x x) 1)"))
(assert (= (LET) None))
(assert (= (LET*) []))

[None, None, None, None, None, None, None, None, None, None, None, None, None, None]

### Do imperative

In [ ]:
; similar to let*
(assert (= (repr* (DO (LET a 1) (LET b 2) (LET c [a b]) [a b c 'd]))  "(1 2 (1 2) d)"))
; do true
(assert (= (str (L (COND (DO TRUE) TRUE FALSE))) "(L a (L b a))"))
; do false
(assert (= (str (L (COND (DO FALSE) TRUE FALSE))) "(L a (L b b))"))

In [ ]:
(print* ((NUM 4) 'a 'b))

In [ ]:
(print (S K S))

In [ ]:
(print ((S K S) 'a))

In [ ]:
(print (SUM ONE TWO))

In [ ]:
(print* ((SUM ONE TWO) 'a 'b))

In [ ]:
(print (MUN THREE))

In [ ]:
(print (MUN (SUM ONE TWO)))

In [ ]:
(print (FACTORIAL FOUR))

In [ ]:
(print* ((FACTORIAL FOUR) 'a 'b))

In [ ]:
(print (MUN (FACTORIAL FOUR)))

In [ ]:
(print* (FIBONACCI2 TEN 'a 'b))

In [ ]:
(print* (MUN (FIBONACCI2 TEN)))

In [ ]:
(print* (MUN (FIBONACCI2 (NUM 14))))

In [ ]:
(print* (MUN (FIBONACCI (NUM 14))))